In [28]:
import numpy as np

def extract_patch(image: np.ndarray, coords, cut_size: int = 9):
    coord_x, coord_y, coord_z = coords
    patch_size = cut_size * 2 + 1

    # Define start and end indices for the patch
    start_x, end_x = max(coord_x - cut_size, 0), min(coord_x + cut_size, image.shape[2] - 1)
    start_y, end_y = max(coord_y - cut_size, 0), min(coord_y + cut_size, image.shape[1] - 1)
    start_z, end_z = max(coord_z - cut_size, 0), min(coord_z + cut_size, image.shape[0] - 1)

    # Initialize the patch with a fill value
    fill_value = -50  # TODO: Adjust based on your data
    patch = np.full((patch_size, patch_size, patch_size), fill_value, dtype=image.dtype)


    # Extract the valid region and place it into the patch
    patch[start_z - coord_z + cut_size:end_z - coord_z + cut_size + 1,
          start_y - coord_y + cut_size:end_y - coord_y + cut_size + 1,
          start_x - coord_x + cut_size:end_x - coord_x + cut_size + 1] = \
          image[start_z:end_z + 1, start_y:end_y + 1, start_x:end_x + 1]

    return patch

In [29]:
arr = np.random.rand(512, 512, 400)
extract_patch(arr, coords=(100, 100, 100), cut_size=9).shape

(19, 19, 19)